In [ ]:
%cd ..

# Azure Machine Learning - Getting Started

##### IMPORTS

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute
from azure.identity import DefaultAzureCredential

from aml.settings import *

Look [here](https://github.com/Azure/azureml-examples/tree/main) for way more information.

## Connect to a Workspace

In [ ]:
credential = DefaultAzureCredential()

# Get a handle to the workspace. You can find the info on the workspace tab on ml.azure.com
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION_ID,  # this will look like xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WORKSPACE_NAME,
)

## Create Compute

Specify attributes in [.env file](../.env)

In [ ]:
create_or_update = False

if COMPUTE_NAME in [com.name for com in ml_client.compute.list()]:
    print(
        f"You already have a cluster named {COMPUTE_NAME}, we'll check whether its attributes match your specifications."
    )
    compute_target = ml_client.compute.get(COMPUTE_NAME)

     # Check if the attributes of the existing compute match the specifications
    differences = []
    if compute_target.type != COMPUTE_TYPE:
        differences.append("type")
    if compute_target.size.upper() != COMPUTE_SIZE.upper():
        differences.append("size")
    if compute_target.min_instances != int(COMPUTE_MIN_INSTANCES):
        differences.append("min_instances")
    if compute_target.max_instances != int(COMPUTE_MAX_INSTANCES):
        differences.append("max_instances")
    if compute_target.idle_time_before_scale_down != float(COMPUTE_IDLE_TIME):
        differences.append("idle_time_before_scale_down")
    if {"low_priority": "LowPriority", "dedicated": "Dedicated"}.get(compute_target.tier) != COMPUTE_TIER:
        differences.append("tier")
        print({"low_priority": "LowPriority", "dedicated": "Dedicated"}.get(compute_target.tier), type({"low_priority": "LowPriority", "dedicated": "Dedicated"}.get(compute_target.tier)), COMPUTE_TIER)

    # Print the differences, if any
    if differences:
        print(f"The following attributes of compute target are different from your specifications: {', '.join(differences)}")
        create_or_update = True
    else:
        print("All attributes of compute_target match the specifications.")
else:
    create_or_update = True

if create_or_update:
    user_input = input("-> Are you sure you want to create/update this Compute? [yes| no]: ")
    print(f"-> Are you sure you want to create/update this Compute? [yes| no]: {user_input.lower()}")
    
    if user_input.upper() == "YES":
        print("Creating/Updating compute target...")
        compute_target = AmlCompute(
            name=COMPUTE_NAME,
            type=COMPUTE_TYPE,
            size=COMPUTE_SIZE,
            min_instances=COMPUTE_MIN_INSTANCES,
            max_instances=COMPUTE_MAX_INSTANCES,
            idle_time_before_scale_down=COMPUTE_IDLE_TIME,
            tier=COMPUTE_TIER,
        )
        compute_target = ml_client.begin_create_or_update(compute_target)
        print(f"AMLCompute with name {compute_target.name} is created/updated, the compute size is {compute_target.size}")
    else:
        print("No compute target created/updated.")